<a href="https://colab.research.google.com/github/tlenzmeier58/big_data_challenge/blob/main/working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import os
 import pandas as pd
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python")
#sys.path.append("/opt/spark-3.2.1-bin-hadoop3.2/python/lib/py4j-0.10.4-src.zip")

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 01:14:48--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.4’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.49MB/s    in 0.2s    

2022-11-29 01:14:48 (4.49 MB/s) - ‘postgresql-42.2.9.jar.4’ saved [914037/914037]



In [ ]:
# Import modules
from pyspark import SparkFiles

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


In [ ]:
from posixpath import sep
#Load data
base_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"
file = "amazon_reviews_us_Watches_v1_00.tsv.gz"
url = base_url + file
spark.sparkContext.addFile(url)

# Initial DF
df = spark.read.option('header', 'true').csv(SparkFiles.get(file),
                                             inferSchema=True,
                                             sep='\t',
                                             timestampFormat = "yyyy-mm-dd")
df.head()

Row(marketplace='US', customer_id=3653882, review_id='R3O9SGZBVQBV76', product_id='B00FALQ1ZC', product_parent=937001370, product_title='Invicta Women\'s 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch', product_category='Watches', star_rating=5, helpful_votes=0, total_votes=0, vine='N', verified_purchase='Y', review_headline='Five Stars', review_body='Absolutely love this watch! Get compliments almost every time I wear it. Dainty.', review_date=datetime.datetime(2015, 1, 31, 0, 8))

In [ ]:
# Cleaning time . . .
df = df.dropna()
df = df.dropDuplicates()

In [ ]:
# Row count
df.count()

960872

In [ ]:
#Schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
 # IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = 'big-data-challenge.ch9chhy09aqn.us-west-2.rds.amazonaws.com' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'big_data_etl' # This is my value; please replace with your own
my_aws_username = 'postgres' # Your value is likely the same, but please double check
my_aws_password = 'LyNnE3055!' # This is my value; please replace with your own

 # Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

In [ ]:
 # Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}


mode = 'overwrite' 

In [ ]:
# Watches df
watches = df[['product_id', 'product_parent', 'product_title']]
watches.head()



# Push to PostgreSQL

watches.write.jdbc(url = jdbc_url,
                   table = 'watches',
                  mode = mode,
                 properties = config)


Row(product_id='B00FALQ1ZC', product_parent=937001370, product_title='Invicta Women\'s 15150 "Angel" 18k Yellow Gold Ion-Plated Stainless Steel and Brown Leather Watch')

In [ ]:
# Reviews df
watch_reviews = df = df[['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date']]
watch_reviews.head()

watch_reviews.write.jdbc(url=jdbc_url,
                   table = 'watch_reviews',
                   mode = mode,
                   properties = config)

In [ ]:
# Customers
watch_customers = df[['customer_id']]
watch_customers = watch_customers.groupBy('customer_id').count()
watch_customers = watch_customers.withColumnRenamed('count','customer_count')
watch_customers.head()

# Push to PostgreSQL
watch_customers.write.jdbc(url=jdbc_url,
                     table = 'watch_customers',
                     mode=mode,
                     properties = config)
